In [408]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


SAMPLE_SPREADSHEET_ID_input = '1QmFZ2epobBFXQ9K0TBUbcPMasiHEjt4DahZ7ue7RuDc'
SAMPLE_RANGE_NAME = 'A1:AA1000'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

main()

df_1=pd.DataFrame(values_input[1:], columns=values_input[0])


In [409]:
hostname = 'lf-prod-replica.cahzexplvjof.eu-west-2.rds.amazonaws.com'
username = 'lf_datascience'
password = 'JHVAZ8Spc0JuQKf2BDu08'
database = 'TheSpaceship'


import psycopg2
import pandas as pd
import pandas.io.sql as sqlio

conn = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
cur = conn.cursor()
auth_cred=('yh2Now9JlNe30cbCFd5g1jTkCk9XqR9O1zQ11KRS', '')

def connection(hostname, username, password, database):
    import psycopg2
    conn = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
    cur = conn.cursor()
    return cur

cur=connection(hostname, username, password, database)


query="""
with loans_info as (

select company_id, sum(amount) amount2 from (
  
  select distinct
  company_id,
 amount::numeric amount
 from term_loans 
   where upper(status)='FUNDED'
 
 ) t
 where  1=1


 group by 1
  )
  ,


data_ as (
   
select 
com.name, com.company_house_no, 
'https://admin.lendflo.com/#/company/'||com.company_id as admin_panel_link, 

case when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 1110 and 3220 then 'Agri Forest&Fishing'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 5101 and 9900 then 'Mining&Quarrying'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 10110 and 33200 then 'Manufacturing'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 35110 and 35300 then 'Electricity Gas Supply'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 36000 and 39000 then 'Water Supply & WasteManagement'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 41100 and 43999 then 'Construction'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 45111 and 47990 then 'WS Retail Trade & RepairMotorV'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 49100 and 53202 then 'Transportation & Storage'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 55100 and 56302 then 'Accommodation & Food'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 58110 and 63990 then 'Information & Communication'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 64110 and 66300 then 'Financial & Insurance'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 68100 and 68320 then 'RealEstate'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 69101 and 75000 then 'Scientific & Technical'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 77110 and 82990 then 'Admin & Support Service'
when (string_to_array(translate(cast(sic_codes as varchar),'[]"',''), ',')) [1]::int between 84110 and 84300 then 'Pub.Admin & Defence'
else 'Others'
end sector ,
sic_codes,
industry sic_description,

cast(com.created_at as date) Application_date,
products.name product, 
funding_limits_view.term, 
--sum(amount2) Drawn_amount,
cast(case when funding_limits_view.funding_limit='' then null else funding_limits_view.funding_limit end as numeric) Granted_amount 
--count(distinct loans_info.company_id) Drawn_count,
--count(distinct funding_limits_view.company_id ) Granted_count



FROM funding_limits_view
join company_products on company_products.company_id= funding_limits_view.company_id and cast(case when company_products.funding_limit='' then null else company_products.funding_limit end as numeric)>1  
    join products on products.id=company_products.product_id 
join companies com on cast(funding_limits_view.company_id as varchar(100))= cast(com.company_id as varchar(100)) --and com.company_status ='ACTIVE'

left join loans_info on cast(loans_info.company_id as varchar(100))= cast(funding_limits_view.company_id as varchar(100))
where 1=1

 --and funding_limits_view.funding_line_atb_id is not null
 and cast(case when funding_limits_view.funding_limit='' then null else funding_limits_view.funding_limit end as numeric) > 1
group by 1,2,3,4,5,6 , 7 , 8 ,9,10

)

select * from data_ where sector='Construction'
order by 5 desc 
            
    """

df_2 = pd.read_sql_query(query,conn)


In [410]:
import datetime
import json
import numpy as np


df_3=df_1[df_1.isnull()] = df_2

def cast_for_gsheets(df):
    for column, dt in zip(df.columns, df.dtypes):
        if dt.type not in [
            np.int64,
            np.float_,
            np.bool_,
        ]:
            df.loc[:, column] = df[column].astype(str)
    return df

df_construction=cast_for_gsheets(df_3)

#change sheet ID
SAMPLE_SPREADSHEET_ID_input = '1QmFZ2epobBFXQ9K0TBUbcPMasiHEjt4DahZ7ue7RuDc'

#change data range
SAMPLE_RANGE_NAME = 'A1:J13'

def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
    global service
    SCOPES = [scope for scope in scopes[0]]
    
    cred = None

    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
            cred = flow.run_local_server()

        with open('token_write.pickle', 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(api_service_name, api_version, credentials=cred)
        print(api_service_name, 'service created successfully')
    except Exception as e:
        print(e)

Create_Service('credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])
    

def Export_Data_To_Sheets():
        response_date = service.spreadsheets().values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
        valueInputOption='RAW',
        range=SAMPLE_RANGE_NAME,
        body=dict(
            majorDimension='ROWS',
            values=(df_construction.T.reset_index().T.values.tolist())) ).execute()
        
print('Sheet successfully Updated')

Export_Data_To_Sheets()

sheets service created successfully
Sheet successfully Updated


In [414]:
$brew install git

SyntaxError: invalid syntax (<ipython-input-414-2eabea4a3b1a>, line 1)